## Contour deformation 

In the context of GW method, contour deformation (CD) technique is used in conjunction with resolution of identity (RI) to reduce the formal scaling of the self-energy calculation. Compared to widely used analytic continuation approach it provides a means to evaluate self-energy  directly on the real axis without employing Pade approximants or non-linear least squares fit and potentially offering superior accuracy. Here, we provide a brief outline of the theory behind CD and give an example of the self-energy calculation within CD without invoking RI in order to facilitate comparison with the results prsented above. 

Detailed discussion of the CD can be found in the following papers:

1. Golze, D., Wilhelm, J., van Setten, M. J., & Rinke, P. (2018). Core-Level Binding Energies from GW : An Efficient Full-Frequency Approach within a Localized Basis. Journal of Chemical Theory and Computation, 14(9), 4856–4869. https://doi.org/10.1021/acs.jctc.8b00458

2. Giantomassi, M., Stankovski, M., Shaltaf, R., Grüning, M., Bruneval, F., Rinke, P., & Rignanese, G.-M. (2011). Electronic properties of interfaces and defects from many-body perturbation theory: Recent developments and applications. Physica Status Solidi (B), 248(2), 275–289. https://doi.org/10.1002/pssb.201046094

CD is used to recast the convolution in the GW expression of self-energy as a difference between two integrals, one which can be performed analytically whereas the other can be evaluated numerically on a relatively small grid. This is achieved by closing the inegration contour as shown below [2]:

![Integration contour used to evaluate $\Sigma(\omega)$](CD_scheme.jpg)

$$
\Sigma(r_1,r_2, \omega) = \frac{i}{2\pi} \int_{-\infty}^{+\infty} e^{i\omega^{\prime} \eta} G(r_1, r_2, \omega + \omega^{\prime}) W(r_1, r_2, \omega^{\prime}) d\omega^{\prime}\\ 
= \frac{i}{2\pi} \oint_{\Gamma} G(r_1, r_2, \omega + z) W(r_1, r_2, z) dz -  \frac{1}{2\pi} \int_{-\infty}^{+\infty} G(r_1, r_2, \omega + i\omega^{\prime}) W(r_1, r_2, i\omega^{\prime}) d\omega^{\prime}
$$

Depending on the $\omega$ value the lower-left and the upper-right loops of the contour can enclose one or several poles of the zero-order Green's function whereas the poles of the screened Coulomb interaction never fall within the contour. This allowes to evaluate the countour integral as a sum of corresponding residues with apropriate signs (note that the upper-right loop is traversed counter-clockwise, while the lower-left loop is traversed clockwise). The imaginary axis contribution is calculated using Gauss-Legendre grid. Importantly, the intgrals over the arches vanish iff the screened Coulomb interaction does not contain the exchange contribution.

In [1]:
import psi4
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML

display(HTML("<style>.container {width:95% !important;}</style>"))


In [3]:

psi4.set_options({'basis' : 'cc-pvdz', 'd_convergence' : 1e-7,'scf_type' : 'out_of_core', 'dft_spherical_points' : 974, 'dft_radial_points' : 150 })

c2h2 = psi4.geometry("""
C 0.0000 0.0000  0.6015
C 0.0000 0.0000 -0.6015
H 0.0000 0.0000  1.6615
H 0.0000 0.0000 -1.6615
                       symmetry c1
                       units angstrom
""")



psi4.set_output_file('c2h2_ccpvdz.out')

scf_e, scf_wfn = psi4.energy('PBE', return_wfn=True)


print("DFT energy is %16.10f" % scf_e)
epsilon = np.asarray(scf_wfn.epsilon_a())
print(epsilon*psi4.constants.hartree2ev)

DFT energy is   -77.2219432464
[-2.69464960e+02 -2.69411046e+02 -1.84233883e+01 -1.39147735e+01
 -1.19976372e+01 -6.91544348e+00 -6.91544341e+00  1.94673225e-01
  1.94673281e-01  1.59118764e+00  2.45849985e+00  7.74384399e+00
  8.70052518e+00  1.16925914e+01  1.16925914e+01  1.47597104e+01
  1.47597105e+01  1.75798680e+01  1.96350562e+01  2.31515682e+01
  2.53626617e+01  2.53626617e+01  2.64280795e+01  2.64280796e+01
  2.77027593e+01  3.27987290e+01  3.60796171e+01  3.60796172e+01
  3.68037399e+01  3.68037400e+01  4.71420823e+01  4.71420823e+01
  4.96327852e+01  5.82212395e+01  6.32069231e+01  6.32069232e+01
  7.69820798e+01  8.95698018e+01]


``` SCF Total Energy (Ha):    -77.2219432068   (MOLGW) ```

In [4]:
import GW

In [5]:
gw_par = {'no_qp' : 7, 'nv_qp' : 0, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 200, 'low_mem' : True }
gw_c2h2_dz_cd1 = GW.GW_DFT(scf_wfn, c2h2, gw_par)
gw_c2h2_dz_cd1.print_summary()

Number of basis functions:  38
occ/virt: 7/31
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  140
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (7, 501)
Caculating GW self-energy via contour deformation
Calculation of the integral term requires    0.064 Gb
Calculation of the residue term requires     0.161 Gb
Using low-memory algorithm
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.30324394 -1.3013649  -0.33389783 -0.23717777 -0.21572427 -0.14005357
 -0.14005358]
Perfoming graphic solution of the inverse Dyson equation
Done!
E^lin, eV  E^graph, eV  Z 
  -286.587831    -326.503140       0.724323
  -286.548967    -326.475725       0.725630
   -22.168472     -21.436808       0.740752
   -17.660116     -17.728666       0.797120
   -16.125265     -15.984765       0.873439
   -10.631349     -10.639259       0.897342
   -10.679617   

```
 GW eigenvalues (eV) RI
   #        E0         SigX-Vxc       SigC          Z        E_qp^lin     E_qp^graph
   1   -269.503377   -35.463486    11.828217     0.724328  -286.623075  -326.542284
   2   -269.449587   -35.412335    11.798952     0.725633  -286.584227  -326.514902
   3    -18.425273    -9.085843     4.032739     0.740744   -22.168328   -21.438530
   4    -13.915903    -6.453950     1.756727     0.797034   -17.659749   -17.729721
   5    -11.997810    -5.869987     1.145594     0.873449   -16.124327   -15.984958
   6     -6.915552    -3.811111    -0.355345     0.897341   -10.654285   -10.639366
   7     -6.915552    -3.811111    -0.355345     0.897341   -10.654285   -10.639366

```

In [6]:
gw_par = {'no_qp' : 7, 'nv_qp' : 0, 'nomega_sigma' : 501, 'step_sigma' : 0.01,  'analytic_W': True, 'gl_npoint' : 200, 'debug' : False, 'low_mem' : False }
gw_c2h2_dz_cd2 = GW.GW_DFT(scf_wfn, c2h2, gw_par)
gw_c2h2_dz_cd2.print_summary()

Number of basis functions:  38
occ/virt: 7/31
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  140
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (7, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
Shape of omega tensor is  (38, 38, 217)
Calculation of the integral term requires    0.064 Gb
Calculation of the residue term requires     6.103 Gb
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.30324394 -1.3013649  -0.33389783 -0.23717777 -0.21572427 -0.14005357
 -0.14005358]
Perfoming graphic solution of the inverse Dyson equation
Done!
E^lin, eV  E^graph, eV  Z 
  -286.589767    -326.503147       0.724323
  -286.550907    -326.475732       0.725630
   -22.169264     -21.436806       0.740752
   -17.660393     -17.728667       0.797120
   -16.125682     -15.984765     

```
Analytic vs approximate W (contour deformation algorithm)

Analytic
E^lin, eV  E^graph, eV  Z 
  -286.589767    -326.503147       0.724323
  -286.550907    -326.475732       0.725630
   -22.169264     -21.436806       0.740752
   -17.660393     -17.728667       0.797120
   -16.125682     -15.984765       0.873439
   -10.631926     -10.639259       0.897342
   -10.680195     -10.639259       0.897342
   
Approximate
E^lin, eV  E^graph, eV  Z 
  -286.587831    -326.503140       0.724323
  -286.548967    -326.475725       0.725630
   -22.168472     -21.436808       0.740752
   -17.660116     -17.728666       0.797120
   -16.125265     -15.984765       0.873439
   -10.631349     -10.639259       0.897342
   -10.679617     -10.639259       0.897342
   
MOLGW reference
GW eigenvalues (eV)
   #        E0         SigX-Vxc       SigC          Z        E_qp^lin     E_qp^graph
   1   -269.503377   -35.463486    11.828217     0.724328  -286.623075  -326.542284
   2   -269.449587   -35.412335    11.798952     0.725633  -286.584227  -326.514902
   3    -18.425273    -9.085843     4.032739     0.740744   -22.168328   -21.438530
   4    -13.915903    -6.453950     1.756727     0.797034   -17.659749   -17.729721
   5    -11.997810    -5.869987     1.145594     0.873449   -16.124327   -15.984958
   6     -6.915552    -3.811111    -0.355345     0.897341   -10.654285   -10.639366
   7     -6.915552    -3.811111    -0.355345     0.897341   -10.654285   -10.639366

```